In [81]:
import pandas as pd
import http.client
import json
import vnstock as vs
from datetime import date
from dateutil.relativedelta import relativedelta

Get all company information

In [82]:
conn = http.client.HTTPSConnection("fiin-core.ssi.com.vn")
payload = ''
headers = {
  'authority': 'fiin-core.ssi.com.vn',
  'accept': 'application/json',
  'accept-language': 'en-US,en;q=0.9',
  'content-type': 'application/json',
  'dnt': '1',
  'origin': 'https://iboard.ssi.com.vn',
  'referer': 'https://iboard.ssi.com.vn/',
  'sec-ch-ua': '"Edge";v="114", "Chromium";v="114", "Not=A?Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'same-site',
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1788.0  uacq',
  'x-fiin-key': 'KEY',
  'x-fiin-seed': 'SEED',
  'x-fiin-user-id': 'ID'
}
conn.request("GET", "/Master/GetListOrganization?language=vi", payload, headers)
res = conn.getresponse()
data = res.read()
# 
# convert data into dataframe with the key 'items'
df_company_list = pd.DataFrame(json.loads(data)['items'])
df_company_list.set_index('ticker', inplace=True)

Set variable

In [83]:
#Declare variable to hold list of company 
ticker_list = list(df_company_list.reset_index()['ticker'])

#Set time for get stock data
start_period = pd.Period(date.today() - relativedelta(months=21), freq='Q-DEC')
end_period = pd.Period(date.today(), freq='Q-DEC')

In [84]:
start_period

Period('2021Q4', 'Q-DEC')

In [85]:
#Get income statement
pd.set_option('display.max_columns', None)

df_concat = []

#Request API and append data
for i in ticker_list:
    try:
        df_data = vs.financial_flow(symbol=i, report_type='incomestatement', report_range='quarterly')[['ticker','postTaxProfit']]
        df_data.index = pd.to_datetime(df_data.index).to_period('Q')
        df_data = df_data[df_data.index >= start_period]
        df_data = df_data[df_data.index <= end_period]
        df_concat.append(df_data)
    except KeyError:
        pass
df_concat = pd.concat(df_concat)
df_incomestatement = df_concat.set_index(df_concat['ticker'], append=True)
del(df_incomestatement['ticker'])


C:\Users\PC\AppData\Local\Temp\ipykernel_29732\2952185138.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_data.index = pd.to_datetime(df_data.index).to_period('Q')
C:\Users\PC\AppData\Local\Temp\ipykernel_29732\2952185138.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_data.index = pd.to_datetime(df_data.index).to_period('Q')
C:\Users\PC\AppData\Local\Temp\ipykernel_29732\2952185138.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_data.index = pd.to_datetime(df_data.index).to_period('Q')
C:\Users\PC\AppData\Local\Temp\ipykernel_29732\2952185138.py:10:

In [86]:
#Create Year on Year and Quarter on Quarter column
df_incomestatement['Quarter On Quarter'] = df_incomestatement.groupby('ticker')['postTaxProfit'].shift(periods=-1)
df_incomestatement['Year on Year'] = df_incomestatement.groupby('ticker')['postTaxProfit'].shift(periods=-4)
df_incomestatement['YoY-ratio'] = (df_incomestatement['postTaxProfit']/df_incomestatement['Year on Year']) - 1

In [87]:
#Add year to date sum column
df_incomestatement.reset_index(inplace=True)
df_incomestatement['RecordYear'] = df_incomestatement['index'].dt.to_timestamp().dt.year
df_incomestatement.set_index(['RecordYear', 'ticker'], inplace=True)
df_incomestatement['YTD postTaxProfit'] = df_incomestatement.groupby(['RecordYear', 'ticker'])[['postTaxProfit']].sum()

In [88]:
test = ','.join(ticker_list)
i = 0

df_marketcap = []
while i <= len(ticker_list):
    try:
        df_data_mark = vs.stock_ls_analysis(','.join(ticker_list[i:i+2]))[['ticker','marcap', 'price']]
        df_marketcap.append(df_data_mark)
        i = i + 2
    except KeyError:
        i = i + 2
        pass

df_marketcap= pd.concat(df_marketcap)
df_marketcap.set_index('ticker', inplace=True)

In [89]:
df_financial = []



for i in ticker_list:
    try:
        df_data = vs.financial_ratio(i, 'quarterly', True)[['quarter','year','ticker', 'roe','priceToBook','earningPerShare', 'priceToEarning']]

        df_data['StockTime'] = df_data['year'].astype('str') +  'Q' + df_data['quarter'].astype('str')
        df_data.set_index('StockTime', inplace=True)
        df_data.index = pd.to_datetime(df_data.index).to_period('Q')
        del(df_data['quarter'])
        del(df_data['year'])
        df_data.set_index(df_data['ticker'], append=True, inplace=True)
        del(df_data['ticker'])

        df_data = df_data.astype({'earningPerShare':float, 'priceToEarning':float})

        df_data = df_data[df_data.index.get_level_values('StockTime') >= start_period]
        df_data = df_data[df_data.index.get_level_values('StockTime') <= end_period]

        df_financial.append(df_data)
    except KeyError:
        pass

df_financial = pd.concat(df_financial)

C:\Users\PC\AppData\Local\Temp\ipykernel_29732\48252454.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_data.index = pd.to_datetime(df_data.index).to_period('Q')
C:\Users\PC\AppData\Local\Temp\ipykernel_29732\48252454.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_data.index = pd.to_datetime(df_data.index).to_period('Q')
C:\Users\PC\AppData\Local\Temp\ipykernel_29732\48252454.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_data.index = pd.to_datetime(df_data.index).to_period('Q')
C:\Users\PC\AppData\Local\Temp\ipykernel_29732\48252454.py:11: UserWar

In [90]:
#Add sum of EPS across 4 latest quarter, EPS/4Q EPS
df_financial_final = df_financial.join(df_financial.reset_index('ticker').groupby('ticker')[['earningPerShare']].rolling(4).sum().shift(-3), rsuffix='-4Q')
df_financial_final['EPS/4Q EPS'] = df_financial_final['earningPerShare'] / df_financial_final['earningPerShare-4Q']

In [91]:
#Joining df_financial_final with df_incomestatement
df_info = df_incomestatement\
    .reset_index(level=0)\
    .rename(columns={'index':'StockTime'})\
    .set_index('StockTime', append=True)\
    .drop(columns='RecordYear')\
    .join(df_financial_final)

In [93]:
#Get a list of ticker base on marketcap
filter_marcap_list =  list(df_marketcap.query("marcap >= 1000").reset_index()['ticker'])

#Filter base on specific stocktime and  ticker_list and other conditions
filter_start_stock_time = pd.Period(date.today() - relativedelta(months=9), freq='Q-DEC')

#Export data
df_info.reset_index().to_excel('raw_data.xlsx')

In [94]:
df_info = pd.read_excel('raw_data.xlsx').set_index(['ticker', 'StockTime'])

df_info

Unnamed: 0  postTaxProfit  Quarter On Quarter  Year on Year  \
ticker StockTime                                                                
VVS    2023Q1              0              1                18.0           NaN   
       2022Q4              1             18                 8.0           NaN   
       2022Q3              2              8                 NaN           NaN   
HSV    2023Q1              3              1                -4.0           4.0   
       2022Q4              4             -4                 2.0           2.0   
...                      ...            ...                 ...           ...   
YEG    2022Q4           6828              9                15.0         281.0   
       2022Q3           6829             15                 7.0           NaN   
       2022Q2           6830              7                 1.0           NaN   
       2022Q1           6831              1               281.0           NaN   
       2021Q4           6832            281                 NaN           NaN   

                  YoY-ratio  YTD postTaxProfit    roe  priceToBook  \
ticker StockTime                                                     
VVS    2023Q1           NaN                  1    NaN          NaN   
       2022Q4           NaN                 26    NaN          NaN   
       2022Q3           NaN                 26    NaN          NaN   
HSV    2023Q1     -0.750000                  1  0.004          0.4   
       2022Q4     -3.000000                  4  0.020          0.3   
...                     ...                ...    ...          ...   
YEG    2022Q4     -0.967972                 32    NaN          NaN   
       2022Q3           NaN                 32    NaN          NaN   
       2022Q2           NaN                 32  0.281          0.8   
       2022Q1           NaN                 32  0.074          1.0   
       2021Q4           NaN                281  0.019          0.9   

                  earningPerShare  priceToEarning  earningPerShare-4Q  \
ticker StockTime                                                        
VVS    2023Q1                 NaN             NaN                 NaN   
       2022Q4                 NaN             NaN                 NaN   
       2022Q3                 NaN             NaN                 NaN   
HSV    2023Q1                37.0           117.8              1388.0   
       2022Q4               213.0            16.4                 NaN   
...                           ...             ...                 ...   
YEG    2022Q4                 NaN             NaN                 NaN   
       2022Q3                 NaN             NaN                 NaN   
       2022Q2              7044.0             3.2                 NaN   
       2022Q1              2019.0            13.8                 NaN   
       2021Q4               534.0            47.7                 NaN   

                  EPS/4Q EPS  
ticker StockTime              
VVS    2023Q1            NaN  
       2022Q4            NaN  
       2022Q3            NaN  
HSV    2023Q1       0.026657  
       2022Q4            NaN  
...                      ...  
YEG    2022Q4            NaN  
       2022Q3            NaN  
       2022Q2            NaN  
       2022Q1            NaN  
       2021Q4            NaN  

[6833 rows x 12 columns]

In [95]:

df_info_filter = df_info[df_info.index.get_level_values('ticker').isin(filter_marcap_list)]\
    .query("`YoY-ratio` > 0.35 & earningPerShare > 800 & `EPS/4Q EPS` > 0.3")\
    .reset_index()
    

df_info_filter

,ticker,StockTime,Unnamed: 0,postTaxProfit,Quarter On Quarter,Year on Year,YoY-ratio,YTD postTaxProfit,roe,priceToBook,earningPerShare,priceToEarning,earningPerShare-4Q,EPS/4Q EPS
0,SSH,2023Q1,38,270,119.0,53.0,4.094340,270,0.112,5.6,1221.0,52.7,3588.0,0.340301
1,ACV,2023Q1,182,1636,1287.0,875.0,0.869714,1636,0.189,3.7,3620.0,21.0,11063.0,0.327217
2,ACV,2022Q4,183,1287,2398.0,333.0,2.864865,7128,0.175,4.2,3271.0,26.0,7758.0,0.421629
3,ANV,2022Q4,289,107,120.0,54.0,0.981481,675,0.258,1.0,5060.0,4.5,15418.0,0.328188
4,APF,2022Q4,301,155,87.0,78.0,0.987179,406,0.424,1.3,14129.0,3.6,42839.0,0.329816
5,BHN,2022Q4,506,52,236.0,1.0,51.000000,528,0.111,2.3,2109.0,21.8,7016.0,0.300599
6,BID,2022Q4,518,4262,5354.0,2296.0,0.856272,18454,0.199,1.9,3596.0,10.7,11643.0,0.308855
7,BMP,2023Q1,594,281,248.0,127.0,1.212598,281,0.319,2.6,10378.0,8.8,30163.0,0.344064
8,BMP,2022Q4,595,248,175.0,114.0,1.175439,695,0.283,1.8,8505.0,6.6,22933.0,0.370863
9,DTP,2023Q1,1059,39,37.0,6.0,5.500000,39,0.264,2.1,9244.0,9.0,28306.0,0.326574


In [97]:
df_final = df_info_filter\
    .set_index('ticker')\
    .join(df_marketcap)\
    .join(df_company_list[['organShortName']])

df_final

,StockTime,Unnamed: 0,postTaxProfit,Quarter On Quarter,Year on Year,YoY-ratio,YTD postTaxProfit,roe,priceToBook,earningPerShare,priceToEarning,earningPerShare-4Q,EPS/4Q EPS,marcap,price,organShortName
ticker,,,,,,,,,,,,,,,,
ACV,2023Q1,182,1636,1287.0,875.0,0.869714,1636,0.189,3.7,3620.0,21.0,11063.0,0.327217,165237,75903,Cảng Hàng không VN
ACV,2022Q4,183,1287,2398.0,333.0,2.864865,7128,0.175,4.2,3271.0,26.0,7758.0,0.421629,165237,75903,Cảng Hàng không VN
ANV,2022Q4,289,107,120.0,54.0,0.981481,675,0.258,1.0,5060.0,4.5,15418.0,0.328188,4932,37050,Thủy sản Nam Việt
APF,2022Q4,301,155,87.0,78.0,0.987179,406,0.424,1.3,14129.0,3.6,42839.0,0.329816,1618,62383,Nông sản Quảng Ngãi
BHN,2022Q4,506,52,236.0,1.0,51.000000,528,0.111,2.3,2109.0,21.8,7016.0,0.300599,10199,44000,HABECO
BID,2022Q4,518,4262,5354.0,2296.0,0.856272,18454,0.199,1.9,3596.0,10.7,11643.0,0.308855,224346,44350,BIDV
BMP,2023Q1,594,281,248.0,127.0,1.212598,281,0.319,2.6,10378.0,8.8,30163.0,0.344064,7490,91500,Nhựa Bình Minh
BMP,2022Q4,595,248,175.0,114.0,1.175439,695,0.283,1.8,8505.0,6.6,22933.0,0.370863,7490,91500,Nhựa Bình Minh
DHT,2023Q1,1425,27,36.0,20.0,0.350000,27,0.135,2.4,1391.0,18.9,4626.0,0.300692,1945,26300,Dược phẩm Hà Tây


In [98]:
df_final['Price / PriceToBook'] = df_final['price'] / df_final['priceToBook']
df_final['Predicted Price'] = (df_final['earningPerShare-4Q'] * df_final['Price / PriceToBook'] * 22.5)**(1/2)
df_final['Diff-ratio predicted vs current price'] = (df_final['Predicted Price']  / df_final['price']) - 1
df_final.sort_values('Diff-ratio predicted vs current price', ascending = False, inplace=True)

In [99]:
df_final.reset_index().to_excel('final.xlsx',index=False)